In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', 1000)
# pd.set_option('display.max_rows', 100)

In [2]:
df_cartoes = pd.read_csv("/media/guilherme/ssd_m2_data/py_new_projects/pred_soccer/notebooks/analises/dados_originais/campeonato-brasileiro-cartoes.csv")
df_estatisticas = pd.read_csv("/media/guilherme/ssd_m2_data/py_new_projects/pred_soccer/notebooks/analises/dados_originais/campeonato-brasileiro-estatisticas-full.csv")
df_partidas = pd.read_csv("/media/guilherme/ssd_m2_data/py_new_projects/pred_soccer/notebooks/analises/dados_originais/campeonato-brasileiro-full.csv")
df_gols = pd.read_csv("/media/guilherme/ssd_m2_data/py_new_projects/pred_soccer/notebooks/analises/dados_originais/campeonato-brasileiro-gols.csv")

In [3]:
df_partidas["data"] = pd.to_datetime(df_partidas["data"], dayfirst=True)
df_partidas = df_partidas[df_partidas["data"].dt.year > 2014]

In [4]:
df_partidas.isna().sum()

ID                    0
rodata                0
data                  0
hora                  0
mandante              0
visitante             0
formacao_mandante     4
formacao_visitante    4
tecnico_mandante      2
tecnico_visitante     1
vencedor              0
arena                 0
mandante_Placar       0
visitante_Placar      0
mandante_Estado       0
visitante_Estado      0
dtype: int64

In [5]:
df_partidas.head()

,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
4986,4987,1,2015-05-09,18:32,Chapecoense,Coritiba,4-2-3-1,3-5-2,V. Eutrópio,M. dos Santos Gonçalves,Chapecoense,Arena Condá,2,1,SC,PR
4987,4988,1,2015-05-09,18:33,Palmeiras,Atletico-MG,4-2-3-1,4-2-3-1,O. de Oliveira Filho,L. Culpi,-,Allianz Parque,2,2,SP,MG
4988,4989,1,2015-05-09,21:00,Fluminense,Joinville,4-3-1-2,4-4-1-1,S. Drubscky de Campos,H. Maria,Fluminense,Maracanã,1,0,RJ,SC
4989,4990,1,2015-05-10,11:00,Gremio,Ponte Preta,4-2-3-1,4-1-2-1-2,L. Scolari,A. Ferreira,-,Arena do Grêmio,3,3,RS,SP
4990,4991,1,2015-05-10,16:00,Athletico-PR,Internacional,4-2-3-1,4-2-3-1,M. Mendes,D. Aguirre Camblor,Athletico-PR,Estádio Joaquim Américo Guimarães,3,0,PR,RS


In [6]:
df_gols.head(3)

,partida_id,rodata,clube,atleta,minuto,tipo_de_gol
0,4607,1,Fluminense,Fred,45,Penalty
1,4607,1,Fluminense,Nirley da Silva Fonseca,59,Gol Contra
2,4607,1,Fluminense,Rafael Sóbis,31,NaN


In [7]:
# df_partidas

In [ ]:
# df_gols


# total_gols
# count_gols_contra
# count_gols_penalty

In [12]:
# Filtrando os minutos que possuem "+"
df_gols_plus = df_gols[df_gols["minuto"].str.contains("+", regex=False)].copy()

# Convertendo os minutos para somar
df_gols_plus["minuto"] = df_gols_plus["minuto"].apply(lambda x: sum(map(int, x.split("+"))))

df_gols_plus

,partida_id,rodata,clube,atleta,minuto,tipo_de_gol
29,4622,2,Sao Paulo,Antônio Carlos dos Santos Aguiar,92,NaN
53,4632,3,Internacional,Charles Aránguiz,46,NaN
86,4646,4,Bahia,Pará,91,NaN
87,4640,4,Chapecoense,Tiago Luís Martins,92,NaN
91,4648,5,Cruzeiro,Ricardo Goulart,46,NaN
...,...,...,...,...,...,...
7961,8016,38,Avai,Marcio Antonio de Sousa Junior,50,NaN
7962,8018,38,Ceara,Vitor Mendes,47,Gol Contra
7967,8025,38,Sao Paulo,Juan Santos da Silva,91,NaN
7974,8017,38,Athletico-PR,Erick Luis Conrado Carvalho,94,NaN


In [ ]:
# Definindo os intervalos de minutos para os gols
intervalos = {
    "gol_no_1_15": (1, 15),
    "gol_no_16_30": (16, 30),
    "gol_no_31_45": (31, 45),
    "gol_no_46_60": (46, 60),
    "gol_no_61_75": (61, 75),
    "gol_no_76_90": (76, 90),
    "gol_no_91_105": (91, 105)
}

# Função para verificar se o minuto está dentro de um intervalo
def contar_gols_por_intervalo(minutos, intervalo):
    return sum(intervalo[0] <= minuto <= intervalo[1] for minuto in minutos)

# Aplicando a lógica de agregação
df_gols_agrupados = df_gols_plus.groupby(["partida_id", "clube", "tipo_de_gol"]).agg(
    **{col: ("minuto", lambda x, intervalo=intervalo: contar_gols_por_intervalo(x, intervalo)) for col, intervalo in intervalos.items()}
).reset_index()

# Criando as colunas com base no tipo_de_gol
df_gols_agrupados["count_gol_penalt"] = (df_gols_agrupados["tipo_de_gol"] == "Penalty").astype(int)
df_gols_agrupados["count_gol_contra"] = (df_gols_agrupados["tipo_de_gol"] == "Gol Contra").astype(int)

# Exibindo as primeiras linhas para verificar
df_gols_agrupados

,partida_id,clube,tipo_de_gol,gol_no_1_15,gol_no_16_30,gol_no_31_45,gol_no_46_60,gol_no_61_75,gol_no_76_90,gol_no_91_105
0,4721,Internacional,Penalty,0,0,0,1,0,0,0
1,4844,Figueirense,Penalty,0,0,0,1,0,0,0
2,4891,Athletico-PR,Penalty,0,0,0,1,0,0,0
3,4922,Botafogo-RJ,Gol Contra,0,0,0,0,0,0,1
4,5002,Sport,Penalty,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
129,7917,Santos,Gol Contra,0,0,0,0,0,0,1
130,7921,Flamengo,Penalty,0,0,0,1,0,0,0
131,7942,Atletico-MG,Penalty,0,0,0,0,0,0,1
132,7957,America-MG,Penalty,0,0,0,0,0,0,1


,partida_id,clube,tipo_de_gol,gol_no_1_15,gol_no_16_30,gol_no_31_45,gol_no_46_60,gol_no_61_75,gol_no_76_90,gol_no_91_105,count_gol_penalt,count_gol_contra
0,4721,Internacional,Penalty,0,0,0,1,0,0,0,1,0
1,4844,Figueirense,Penalty,0,0,0,1,0,0,0,1,0
2,4891,Athletico-PR,Penalty,0,0,0,1,0,0,0,1,0
3,4922,Botafogo-RJ,Gol Contra,0,0,0,0,0,0,1,0,1
4,5002,Sport,Penalty,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
129,7917,Santos,Gol Contra,0,0,0,0,0,0,1,0,1
130,7921,Flamengo,Penalty,0,0,0,1,0,0,0,1,0
131,7942,Atletico-MG,Penalty,0,0,0,0,0,0,1,1,0
132,7957,America-MG,Penalty,0,0,0,0,0,0,1,1,0


In [ ]:
# Criando as contagens para gols de pênalti e gols contra
df_gols = df_gols.groupby(["tipo_de_gol"])["tipo_de_gol"].count().unstack(fill_value=0)

# Renomeando colunas para um padrão claro
df_gols = df_gols.rename(columns={"Penalty": "count_gol_penalt", "Gol Contra": "count_gol_contra"})

# Garantindo que as colunas existam, mesmo que não haja registros
df_gols = df_gols.reindex(columns=["count_gol_penalt", "count_gol_contra"], fill_value=0)


In [ ]:
df_gols.head(3)

In [ ]:
# Lista das colunas desejadas
colunas_desejadas = [
    "count_gol_penalt", "count_gol_contra"
]

# Removendo os % diretamente nas colunas

# Renomear as colunas para facilitar o merge
df_gols_home = df_gols.rename(columns={col: f"home_{col}" for col in colunas_desejadas})
df_gols_away = df_gols.rename(columns={col: f"away_{col}" for col in colunas_desejadas})

df_partidas = df_partidas.merge(
    df_gols_home, 
    left_on=["ID", "mandante"], 
    right_on=["partida_id", "clube"], 
    how="left"
).drop(columns=["partida_id", "clube"])

In [ ]:
import pandas as pd

# Lista das colunas desejadas
colunas_desejadas = [
    "chutes", "chutes_no_alvo", "posse_de_bola_%", "passes", "precisao_passes_%",
    "faltas", "cartao_amarelo", "cartao_vermelho", "impedimentos", "escanteios"
]

# Removendo os % diretamente nas colunas
df_estatisticas["posse_de_bola_%"] = df_estatisticas["posse_de_bola"].astype(str).str.replace("%", "")
df_estatisticas["precisao_passes_%"] = df_estatisticas["precisao_passes"].astype(str).str.replace("%", "")
df_estatisticas.drop(columns=["rodata", "posse_de_bola", "precisao_passes"], inplace=True)

# Renomear as colunas para facilitar o merge
df_est_home = df_estatisticas.rename(columns={col: f"home_{col}" for col in colunas_desejadas})
df_est_away = df_estatisticas.rename(columns={col: f"away_{col}" for col in colunas_desejadas})

# Mesclar estatísticas do time mandante
df_partidas = df_partidas.merge(
    df_est_home, 
    left_on=["ID", "mandante"], 
    right_on=["partida_id", "clube"], 
    how="left"
).drop(columns=["partida_id", "clube"])

# Mesclar estatísticas do time visitante
df_partidas = df_partidas.merge(
    df_est_away, 
    left_on=["ID", "visitante"], 
    right_on=["partida_id", "clube"], 
    how="left"
).drop(columns=["partida_id", "clube"])


In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Calcula a matriz de correlação
correlation_matrix = df_partidas.select_dtypes(["int", "float"]).drop(columns=["ID"]).corr()

# Configura o tamanho do gráfico
plt.figure(figsize=(12, 8))

# Cria um heatmap com seaborn
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)

# Adiciona título
plt.title("Matriz de Correlação", fontsize=16)

# Exibe o gráfico
plt.show()

In [ ]:
df_estatisticas.tail() end

In [ ]:
drop_columns_estatisticas = ["rodata"]
df_merge = pd.merge(df_partidas, df_estatisticas, how='left', left_on="ID", right_on="partida_id")

df_merge

In [ ]:


last_id = df_partidas["ID"].iloc[0]
df_partidas.head()

In [ ]:
df_estatisticas = df_estatisticas[df_estatisticas["partida_id"]>=last_id].copy()

In [ ]:


# Existe alguma relação entre posse de bola e vitoria?

In [ ]:
df_estatisticas["posse_de_bola"] = df_estatisticas["posse_de_bola"].astype(str)
df_estatisticas["posse_de_bola_%"] = df_estatisticas["posse_de_bola"].apply(lambda x: x.replace("%", ''))

df_estatisticas.head()

In [ ]:
# Qual a relação entre chutes e numeros de gols?

df_partidas

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_plot = df_estatisticas[df_estatisticas["clube"].isin(["Fluminense"])].copy()
df_plot.sort_values("posse_de_bola_%", inplace=True)

df_plot.dropna(inplace=True)

plt.figure(figsize=(15,6))
plt.title("Relação entre posse_de_bola_% vs passes")
plt.scatter(x=df_plot["passes"], y=df_plot["posse_de_bola_%"])
plt.ylabel("posse_de_bola_%")
plt.xlabel("passes")

In [ ]:
df_plot = df_estatisticas[df_estatisticas["clube"].isin(["Chapecoense"])].copy()
df_plot.sort_values("posse_de_bola_%", inplace=True)

df_plot.dropna(inplace=True)

plt.figure(figsize=(15,6))
plt.title("Relação entre posse_de_bola_% vs chutes")
plt.scatter(x=df_plot["chutes"], y=df_plot["posse_de_bola_%"])
plt.ylabel("posse_de_bola_%")
plt.xlabel("chutes")

In [ ]:
import sqlite3
import pandas as pd
import os
from pathlib import Path


def create_connection(db_path):
    """Cria a conexão com o banco de dados SQLite."""
    connection = None
    try:
        connection = sqlite3.connect(db_path)

        print("Conexão SQLite estabelecida.")
    except sqlite3.Error as e:
        print(f"Erro ao conectar ao SQLite: {e}")
    return connection

BASE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
DATA_DIR = os.path.join(BASE_DIR, 'database')
FT_DIR = os.path.join(BASE_DIR, 'modelagem', 'feature_eng', 'data')

conn = create_connection(db_path=Path(os.path.join(DATA_DIR, "soccer_data.db")))
country = "Brazil"
query = "SELECT * FROM soccer_data WHERE country = ?"

df_hist_soccer_matches = pd.read_sql_query(query, conn, params=(country,))

conn.close()


In [ ]:
df_hist_soccer_matches

In [ ]:
df_ft = pd.read_csv("/media/guilherme/ssd_m2_data/py_new_projects/pred_soccer/modelagem/feature_eng/data/ft_df.csv")

In [ ]:
df_ft_2023 = df_ft[df_ft["season"]==2023]

In [ ]:
df_ft_2023["ht_points"]